# Structure:
This notebook generates experimental designs for the MSIT, specifically for the NTNU study



NTNU design for main participants:

no. subs = 72

no. TRs per trial = 5

length trial = 7s

no. trials = 222

prob null trials = 0.1

no. blocks = 2

In [1]:
# Import libraries and set up plotting
from __future__ import division
import nideconv
from nideconv import ResponseFitter
from nideconv import simulate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import scipy as sp
from scipy import signal
from nideconv.utils import convolve_with_function
import itertools
from itertools import combinations
from itertools import permutations
from collections import Counter
import sys
sns.set_style('white')
sns.set_context('notebook')
palette = sns.color_palette('Set1')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) # cell wdith at 80% of window

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 1000

# Ensure correct wokring directory
if os.getcwd().split('/')[-1].startswith('IMCN-MSIT') == True:
    print('In the correct working directory: ' + str(os.getcwd()))
else:
    os.chdir("..")
    print('Changed to correct working directory: ' + str(os.getcwd()))

In the correct working directory: /Users/scotti/surfdrive/Shared/NTNU_fMRI_tasks/tasks_for_at_NTNU/IMCN-MSIT


# Notes
Blocked design: better for detection of activate areas assuming a certain HRF shape.
Event related design: useful to estimate the shape of the HRF

# Generate onsets and designs for MSIT

function to generate possible conditions for MSIT

In [2]:
# function to generate possible conditions for MSIT
def gen_conditions(verbose=True):
    # make all possible conditions
    cond1 = np.array(['100','020','003']) # condition 1 = control (less flanker & no simon)
    cond2 = np.array(['001','010','002','200','300','030']) # condition 2 = simon interference
    cond3 = np.array(['122','133','121','323','113','223']) # condition 3 = simon control with flanker
    cond4 = np.array(['212','313','221','331','211','233','112','332','311','322','131','232']) # condition 4 = flanker & simon full interference 

    # show all possible combinations of sequences, 3 items per condition
    comb1 = cond1 # only 1 possible combination
    comb2 = list(combinations(cond2,3)) # 20 possible combinations
    comb3 = list(combinations(cond3,3)) # 20 possible combinations
    comb4 = list(combinations(cond4,3)) # 220 possible combinations

    # Restrict each condition further:
    # each sequence set for each participant must include the responses 1, 2 & 3 as options 
    # e.g. cannot have 112, 233, 131 as a sequence set for a participant 
    # The following code therefore loops through all combinations and only keeps ones that have a stimulus
    # from all each category of 'one', 'two' or 'three' as responses.

    # condition 4
    cond41 = cond4[range(0,4)] # stimuli where unique = 1
    cond42 = cond4[range(4,8)] # stimuli where unique = 2
    cond43 = cond4[range(8,12)] # stimuli where unique = 3

    comb4final = []
    for i in comb4:
        cond_one =  any(elem in cond41  for elem in i)
        cond_two =  any(elem in cond42  for elem in i)
        cond_three =  any(elem in cond43  for elem in i)   

        if cond_one and cond_two and cond_three == True:
            comb4final.append(i)

    # condition 3
    cond31 = cond3[range(0,2)] # stimuli where unique = 1
    cond32 = cond3[range(2,4)] # stimuli where unique = 2
    cond33 = cond3[range(4,6)] # stimuli where unique = 3

    comb3final = []
    for i in comb3:
        cond_one =  any(elem in cond31  for elem in i)
        cond_two =  any(elem in cond32  for elem in i)
        cond_three =  any(elem in cond33  for elem in i)   

        if cond_one and cond_two and cond_three == True:
            comb3final.append(i)

    # condition 2
    cond21 = cond2[range(0,2)] # stimuli where unique = 1
    cond22 = cond2[range(2,4)] # stimuli where unique = 2 
    cond23 = cond2[range(4,6)] # stimuli where unique = 3

    comb2final = []
    for i in comb2:
        cond_one =  any(elem in cond21  for elem in i)
        cond_two =  any(elem in cond22  for elem in i)
        cond_three =  any(elem in cond23  for elem in i)   

        if cond_one and cond_two and cond_three == True:
            comb2final.append(i)
            
    if verbose:
        print('Condition 1 options: ' + str(cond1))
        print('Condition 2 options: ' + str(comb2final))
        print('Condition 3 options: ' + str(comb3final))
        print('Condition 4 options: ' + str(comb4final))
        
    return cond1, comb2final, comb3final, comb4final

Function to generate designs for each participant

In [3]:
# # Generate design 
# # run randomized non-block deisgn, inputs required are number of blocks, number of trials, 
# # number of conditions etc
# # columns: sub, trialnum, block, cond, stimulus, identity, jitter, onset, 
# # Trials per condition must be divisible by 3

# def gen_onsets_and_design(TR=1, nTRs=None, nSub=1, nConds=4, nBlocks=5, nTrialBlocks=180,
#                           jitters=None, cond1=None, comb2final=None, 
#                           comb3final=None, comb4final=None, verbose=True):

#     nSub=nSub                                # subject number
#     nBlocks=nBlocks                          # number of mini blocks per condition
#     nConds=nConds                            # number of conditions
#     jitters=jitters                          # possible jitter times
#     nTrialBlocks=nTrialBlocks                # How many trials per block
#     nTotaltrials = nTrialBlocks * nBlocks    # total number of trials
#     stim_dur = 1.6                           # stimulus duration
#     timing_dur = 0.4                         # timing feedback duration
#     feed_dur = 0.0                           # accuracy feedback duration
#     nTRs = nTRs                              # number of TRs per trial
#     trial_len = nTRs*TR                      # Length of the trial 
    
#     while nTrialBlocks % 12 != 0:
#         print(f'Trials per block should be divisible by 12 and 4. The inputted number: {nTrialBlocks}, is not')
#         nTrialBlocks = input('How many trials do you want? ')
    
#     con = list(cond1)                 # stimuli for condition 1
#     sim = list(comb2final[nSub%8])    # stimulit for condition 2
#     flank = list(comb3final[nSub%8])  # stimuli for condition 3
#     inc = list(comb4final[nSub%64])    # stimulu for condition 4
        
#     all_stim = (con + sim + flank + inc)*int(nTotaltrials/12) # all stimuli for this subject
#     this_cond = np.tile(np.repeat([1,2,3,4],3),int(nTotaltrials/12)) # condition corresponding to stimulus
        
#     this_block = np.repeat(range(1,nBlocks+1),nTrialBlocks) # block number
        
#     jittery = np.tile(np.repeat(jitters,int(nTrialBlocks/len(jitters))),nBlocks)
#     #jittery = np.repeat(jitters,nTrialBlocks/len(jitters))
        
#     design = pd.DataFrame({'subject': nSub,                    # subject number
#                             'stimuli': all_stim,                # stimulus presented
#                             'condition': this_cond,              # condition
#                             'jitter': jittery,                   # jitter time
#                             'unique': np.nan,                    # correct response
#                             'block': this_block,                 # block number
#                             'fix_circ_1': jittery,                # duration of first fixation circle
#                             'stimulus_dur': stim_dur,              # duration of stimulus
#                             'feedback_cor_dur': feed_dur,          # duration of correctness feedback
#                             'feedback_timing_dur': timing_dur,      # duratinos of timing feedback
#                             'ITI': (trial_len-timing_dur-stim_dur-feed_dur) - jittery,  # duration of third fixation circle,
#                             'null_trial': 0,                    # add null trial column
#                             'n_trs': nTRs                       # how many TRs per trial?
#                           })
        
#     # add correct response to identity column
#     for i, row in design.iterrows():
#         counter = Counter(design.loc[i,'stimuli'])
#         unique = min(counter, key=counter.get) # find correct answer for each stimulus  
#         design.loc[i, 'unique'] = unique # display correct answer in dataframe
        
#     return design


In [75]:
# Generate design 
# run randomized non-block deisgn, inputs required are number of blocks, number of trials, 
# number of conditions etc
# columns: sub, trialnum, block, cond, stimulus, identity, jitter, onset, 
# Trials per condition must be divisible by 3

def gen_onsets_and_design(TR=1, nTRs=None, nSub=1, nConds=4, nBlocks=5, nTrialBlocks=180,
                          jitters=None, cond1=None, comb2final=None, 
                          comb3final=None, comb4final=None, verbose=True):

    nSub=nSub                                # subject number
    nBlocks=nBlocks                          # number of mini blocks per condition
    nConds=nConds                            # number of conditions
    jitters=jitters                          # possible jitter times
    nTrialBlocks=nTrialBlocks                # How many trials per block
    nTotaltrials = nTrialBlocks * nBlocks    # total number of trials
    stim_dur = 1.6                           # stimulus duration
    timing_dur = 0.4                         # timing feedback duration
    feed_dur = 0.0                           # accuracy feedback duration
    nTRs = nTRs                              # number of TRs per trial
    trial_len = nTRs*TR                      # Length of the trial 

    con = list(cond1)                 # stimuli for condition 1
    sim = list(comb2final[nSub%8])    # stimulit for condition 2
    flank = list(comb3final[nSub%8])  # stimuli for condition 3
    inc = list(comb4final[nSub%64])    # stimulu for condition 4    

    if nTrialBlocks % 12 == 4: # then trial number is not divisible by 12
        rem = nTrialBlocks % 12
        new_ntrials = nTrialBlocks - rem
        some_stim = (con + sim + flank + inc)*int(new_ntrials/12) # all stimuli for this subject
        some_cond = np.tile(np.repeat([1,2,3,4],3),int(new_ntrials/12)) # condition corresponding to stimulus
        all_stim = some_stim + list(np.random.choice(con, 1)) + list(np.random.choice(sim,1)) + list(np.random.choice(flank,1)) + list(np.random.choice(inc,1)) 
        this_cond = list(some_cond) + list([1,2,3,4])
    elif nTrialBlocks % 12 == 0: # then trial number is divisble by 12
        all_stim = (con + sim + flank + inc)*int(nTotaltrials/12) # all stimuli for this subject
        this_cond = np.tile(np.repeat([1,2,3,4],3),int(nTotaltrials/12)) # condition corresponding to stimulus
    else:
        print('trial number must be divisible by 12')
        
    this_block = np.repeat(range(1,nBlocks+1),nTrialBlocks) # block number
        
    jittery = np.tile(np.repeat(jitters,int(nTrialBlocks/len(jitters))),nBlocks)
    #jittery = np.repeat(jitters,nTrialBlocks/len(jitters))
        
    all_stim = all_stim * nBlocks
    this_cond = this_cond * nBlocks
        
    design = pd.DataFrame({'subject': nSub,                    # subject number
                            'stimuli': all_stim,                # stimulus presented
                            'condition': this_cond,              # condition
                            'jitter': jittery,                   # jitter time
                            'unique': np.nan,                    # correct response
                            'block': this_block,                 # block number
                            'fix_circ_1': jittery,                # duration of first fixation circle
                            'stimulus_dur': stim_dur,              # duration of stimulus
                            'feedback_cor_dur': feed_dur,          # duration of correctness feedback
                            'feedback_timing_dur': timing_dur,      # duratinos of timing feedback
                            'ITI': (trial_len-timing_dur-stim_dur-feed_dur) - jittery,  # duration of third fixation circle,
                            'null_trial': 0,                    # add null trial column
                            'n_trs': nTRs                       # how many TRs per trial?
                          })
        
    # add correct response to identity column
    for i, row in design.iterrows():
        counter = Counter(design.loc[i,'stimuli'])
        unique = min(counter, key=counter.get) # find correct answer for each stimulus  
        design.loc[i, 'unique'] = unique # display correct answer in dataframe
        
    return design


Check both functions work

In [73]:
# compuate all possile combinations of conditions
cond1, comb2final, comb3final, comb4final = gen_conditions(verbose=False)

#jitters=[0.33,0.67,1.45,2.9]
jitters=[0.5,1.0,1.5,2.0]
des = gen_onsets_and_design(TR=2.5, nTRs=5, nSub=1, nConds=4, nBlocks=3, nTrialBlocks=100,
                          jitters=jitters, cond1=cond1, comb2final=comb2final, 
                          comb3final=comb3final, comb4final=comb4final, verbose=True)

In [74]:
# check outputted deisgn
des.tail(5)

,subject,stimuli,condition,jitter,unique,block,fix_circ_1,stimulus_dur,feedback_cor_dur,feedback_timing_dur,ITI,null_trial,n_trs
295,1,322,4,2.0,3,3,2.0,1.6,0.0,0.4,8.5,0,5
296,1,020,1,2.0,2,3,2.0,1.6,0.0,0.4,8.5,0,5
297,1,030,2,2.0,3,3,2.0,1.6,0.0,0.4,8.5,0,5
298,1,122,3,2.0,1,3,2.0,1.6,0.0,0.4,8.5,0,5
299,1,212,4,2.0,1,3,2.0,1.6,0.0,0.4,8.5,0,5


# Define other important functions

In [76]:
# Add in pseudorandomizer class
class Pseudorandomizer(object):
    
    #def __init__(self, data, max_identical_iters={'cue': 4, 'correct_answer': 4}):  
    def __init__(self, data, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 4}):  
        self.data = data
        self.max_identical_iters = {x: y+1 for x, y in list(max_identical_iters.items())}
                                    # add 1: if 4 rows is allowed, only give an error after 5 identical rows
    
    def check_trial_rows(self, data, row_n): 
        """
        Returns True if any of the conditions for pseudorandomization are violated for the given rows, 
        False if they are fine.
        """
        
        # First, check for the maximum iterations
        for column, max_iter in list(self.max_identical_iters.items()):
            if row_n - max_iter < 0:
                continue

            # Select rows [max_iter-1 - row_n] we're going to check. Never select any row with index < 0
            row_selection = [x for x in np.arange(row_n, row_n-max_iter, -1)]

            # Next, we check if the selected rows only contain *1* trial type. 
            # If so, this means we have max_iter rows of the same trials, and we need to change something.
            if data.iloc[row_selection][column].nunique() == 1:
                return True

        return False

    def run(self, debug=False):
        """
        Pseudorandomizes: makes sure that it is not possible to have more than x iterations for every type of column, specified in columns.
        """
        # Start by copying from original data, and shuffle
        self.data = self.data.sample(frac=1, 
                                     random_state=np.random.randint(0, 1e7, dtype='int')).reset_index(drop=True) 
        
        if debug:
            outer_while_i = 0
            debug_print_after_i = 100

        good_set = False
        while not good_set:
            if debug:
                outer_while_i += 1

            reshuffle = False  # Assume the dataset does not need reshuffling.
            for row_n in range(0, self.data.shape[0]):

                # Select rows [max_iter-1 - row_n] we're going to check

                # Check if the current row, and the (max_iters-1) rows before, are the same value (number of unique values = 1).
                # If so, then move the current row number to the bottom of the dataframe. However, we need to re-check the same four rows again
                # after moving a row to the bottom: therefore, a while loop is necessary.
                checked_row = False
                n_attempts_at_moving = 0
                
                if debug:
                    inner_while_i = 0
                
                while not checked_row:
                    if debug:
                        inner_while_i += 1
                        if inner_while_i > debug_print_after_i:
                            print('New inner loop started for current row')

                    if self.check_trial_rows(self.data, row_n):
                        if debug and inner_while_i > debug_print_after_i:
                            print('Found too many consecutively identical rows.')

                        # If there are too many consecutively identical rows at the bottom of the dataframe, 
                        # break and start over/shuffle
                        if row_n >= (self.data.shape[0] - self.max_identical_iters[list(self.max_identical_iters.keys())[0]]):
                            if debug and inner_while_i > debug_print_after_i:
                                print(('These occurred at row_n %d, which is at the bottom of the DF.' % row_n))

                            checked_row = True
                            reshuffle = True

                        # Too many consecutive identical rows? Move row_n to the bottom, and check again with the new row_n.
                        else:
                            if debug and inner_while_i > debug_print_after_i:
                                print(('These occurred at row_n %d. Checking the remainder of the DF.' % row_n))

                            # Check if moving to the bottom even makes sense: if all remaining values are identical, it doesn't.
                            if (self.data.iloc[row_n:][list(self.max_identical_iters.keys())].nunique().values < 2).any():
                                if debug and inner_while_i > debug_print_after_i:
                                    print('All remaining values are identical. I should stop the for-loop, and start over.')

                                checked_row = True
                                reshuffle = True
                            else:
                                if n_attempts_at_moving < 50:
                                    n_attempts_at_moving += 1

                                    if debug and inner_while_i > debug_print_after_i:
                                        print('Not all remaining values are identical. I should move the final part to the bottom.')

                                    # If not, move the current row to the bottom
                                    row_to_move = self.data.iloc[row_n,:]

                                    # Delete row from df
                                    self.data.drop(row_n, axis=0, inplace=True)

                                    # Append original row to end. Make sure to reset index
                                    self.data = self.data.append(row_to_move).reset_index(drop=True)

                                # If we already tried moving the current row to the bottom for 50 times, let's forget about it and restart
                                else:
                                    checked_row = True
                                    reshuffle = True
                    else:
                        if debug and inner_while_i > debug_print_after_i:
                            print('Checked row, but the row is fine. Next row.')
                        checked_row = True

                if reshuffle:
                    good_set = False
                    break  # out of the for loop

                # Reached the bottom of the dataframe, but no reshuffle call? Then we're set.
                if row_n == self.data.shape[0]-1:
                    good_set = True

            if reshuffle:
                # Shuffle, reset index to ensure trial_data.drop(row_n) rows
                self.data = self.data.sample(frac=1, random_state=np.random.randint(0, 1e7, dtype='int')).reset_index(drop=True)
        
        return self.data
    
# def add_pseudorandom_null_trials(data, min_row=4, max_row=4, min_n_rows_separate=7, 
#                                 n_null_trials=10, null_column_name=''):
#     """ 
#     Adds null trials interspersed at pseudorandom locations. You can determine the minimum
#     number of trials at the start before a null trial, the minimum number of trials at the end in which no
#     nulls are shown, and the minimum number of trials that the null trials have to be separated 
#     """
    
#     good_idx = False
#     while not good_idx:
#         indx = np.random.choice(np.arange(min_row, data.shape[0]-max_row), 
#                                 replace=False, size=n_null_trials)
#         diffs = np.diff(np.sort(indx))
#         if (diffs >= min_n_rows_separate).all():
#             good_idx = True
    
#     data.index = np.setdiff1d(np.arange(data.shape[0] + n_null_trials), indx)
#     new_rows = pd.DataFrame({null_column_name: [True]*n_null_trials}, columns=data.columns, index=indx)    
#     data = data.append(new_rows).sort_index()
    
#     # Always end with a null trial
#     last_row = pd.DataFrame({null_column_name: [True]*1}, columns=data.columns, index=[data.shape[0]])
#     data = data.append(last_row).sort_index() 
    
#     return data

def add_pseudorandom_null_trials(data, min_row=4, max_row=4, min_n_rows_separate=7, 
                                n_null_trials=10, null_column_name=''):
    """ 
    Adds null trials interspersed at pseudorandom locations. You can determine the minimum
    number of trials at the start before a null trial, the minimum number of trials at the end in which no
    nulls are shown, and the minimum number of trials that the null trials have to be separated 
    """
    
    print("Adding null trials")
    
    if n_null_trials != 0:
        n_null_trials = n_null_trials-1
    
    good_idx = False
    while not good_idx:
        indx = np.random.choice(np.arange(min_row, data.shape[0]-max_row), 
                                replace=False, size=n_null_trials)
        diffs = np.diff(np.sort(indx))
        if (diffs >= min_n_rows_separate).all():
            good_idx = True
    
    data.index = np.setdiff1d(np.arange(data.shape[0] + n_null_trials), indx)
    new_rows = pd.DataFrame({null_column_name: [True]*n_null_trials}, columns=data.columns, index=indx)    
    data = data.append(new_rows).sort_index()
    
    # Always end with a null trial
    if n_null_trials != 0:
        last_row = pd.DataFrame({null_column_name: [True]*1}, columns=data.columns, index=[data.shape[0]])
        data = data.append(last_row).sort_index() 
    
    print("Finished adding null trials")
    return data

# brute force optimization code to ensure non-correlation between task events
def optimize_brute_force(n_trials, 
                         c,   # contrasts to be optimized
                         run_type='localizer',
                         block_number=0,
                         pseudorandomize=True, 
                         TR=3,
                         n_attempts=1e4,
                         
                         # For non-localizer:
                         n_null_trials=0,
                         response_modality='hand',
                         
                         # for localizer:
                         n_localizer_blocks=None, localizer_order=None):
    
    """ Performs a brute force search of the best possible trial order & jitter times for a single run """
    n_attempts = int(n_attempts)
    
    # Generate n_attempt seeds to check
    seeds = np.round(np.random.uniform(low=int(0), high=int(2**32 - 1), size=n_attempts)).astype(int)
    effs = np.zeros(shape=seeds.shape)
    best_eff = 0
    
    best_block = None
    
    for i in range(n_attempts):
       
        # Set seed
        np.random.seed(seed=seeds[i])
        
        # Generate run
        if run_type == 'localizer':
            block_data = create_localizer(n_localizer_blocks=n_localizer_blocks, 
                                          n_trials_per_localizer_block=n_trials, 
                                          localizer_order=localizer_order, 
                                          block_number=block_number, 
                                          pseudorandomize=pseudorandomize, TR=TR)
        elif run_type == 'cognitive':
            block_data = create_cognitive_block(n_trials=n_trials, 
                                                block_number=block_number, 
                                                response_modality=response_modality,
                                                n_null_trials=n_null_trials, 
                                                TR=TR)
        elif run_type == 'limbic':
            block_data = create_limbic_block(n_trials=n_trials, 
                                             block_number=block_number, 
                                             response_modality=response_modality,
                                             n_null_trials=n_null_trials,
                                             TR=TR)

        # Add trial start times (relative to start of experiment)
        block_data['trial_start_time'] = block_data['trial_duration'].shift(1).cumsum()
        block_data.loc[0, 'trial_start_time'] = 0

        # Add cue onset times (relative to start of experiment)
        block_data['cue_onset_time'] = block_data['trial_start_time'] + \
                                       block_data['phase_1']

        # Add stimulus onset times (relative to start of experiment)
        block_data['stimulus_onset_time'] = block_data['trial_start_time'] + \
                                            block_data['phase_1'] + \
                                            block_data['phase_2'] + \
                                            block_data['phase_3']
        
        # Calculate efficiency
        X, ev_names = stim_to_design(block_data, block=block_number, silent=True)
        
        # Loop over the contrasts
        dvars = [(c[ii, :].dot(np.linalg.pinv(X.T.dot(X))).dot(c[ii, :].T))
                 for ii in range(c.shape[0])]
        eff = c.shape[0] / np.sum(dvars)
        effs[i] = eff
        
        # Found a better block than anything earlier? Save this
        if eff > best_eff:
            best_eff = eff
            best_seed = seeds[i]
            best_block = block_data   
                
    # Save everything
    out_dict = {'seeds': seeds, 
                'efficiencies': effs, 
                'best_eff': best_eff, 
                'best_seed': best_seed, 
                'best_block': best_block,
                'contrasts': c,
                'contrast_ev_names': ev_names}
    
    print(('Done optimizing, tried %d seeds. Best efficiency: %.4f (mean eff: %.3f (SD %.3f))' % 
          (n_attempts, best_eff, np.mean(effs), np.std(effs))))

    # return block
    return best_block, out_dict

# Generate main design for all subjects, with practice session design

Set parameters for main design generation

In [82]:
# Set general variables
pNums = 72
TR = 1.4
nTRs = 5
n_blocks = 2              # number of blocks
nConds = 4
n_sessions = 1          # number of sessions
stim_dur = 1.6         # stimulis presentation duration
timing_dur = 0.4         # timing feedback duration
feed_dur = 0.0          # correctness feedback duration (practice only)

# work out settings
block_length = None
num_trials = 111 # per block
trial_len = nTRs*TR
p_null_trials = 0.1

# Work out what jitters times to use based on TR
jitters = [0.5, 1.0, 1.5, 2.0]

In [83]:
input_type = input('Do you want to set settings based on length of time per block or number of trials (''l'' or ''t'')?: ')

if input_type in ['l','L']:
    block_length = int(input('how long should the block be? (s)'))
    
elif input_type in ['t','T']:
    num_trials = int(input('How many trials should be in the block? '))
    
if block_length is not None: # if they have inputted length of block
    print(f'Each block will be {block_length} seconds long ({block_length/60.0} mins)')
    if (block_length%trial_len) != 0:
        print(f"Length of block needs to be divisible by {trial_len}, {block_length} is not")
        raise
    b_len = block_length
    n_trials = block_length/trial_len
    
if num_trials is not None: # if they have inputted number of trials per block
    print(f'Each block will contain {num_trials} trials, and last {num_trials*trial_len} seconds ({round((num_trials*trial_len)/60.0,2)} mins)')
    n_trials = num_trials
    b_len = num_trials*trial_len
    
n_null_trials = round(p_null_trials*n_trials)   # number of null trials per block
n_not_null_trials = n_trials - n_null_trials             # number of trials that are not null trials per block
    
print(f'Settings:\n\n\
        Sessions: {n_sessions}\n\
        Blocks per session: {n_blocks}\n\
        Trials per block: {n_trials}\n\
        Trials for each cond per block: {((n_trials-n_null_trials)/nConds)}\n\
        Number of null trials per block: {n_null_trials}\n\
        Assuming a TR of {TR} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {b_len*n_blocks} seconds\n\
        Total duration: {(b_len*n_blocks)/60.0} minutes\n\
        Total volumes: {(n_trials*n_blocks*nTRs)+1} volumes\n\
        Total duration of block: {b_len} seconds = {b_len/60.0} min\n\
        Total number of volumes necessary for 1 block: 1 + {n_trials*nTRs} = {n_trials*nTRs+1} + warm-up pulses\n')

Do you want to set settings based on length of time per block or number of trials (l or t)?: t
How many trials should be in the block? 111
Each block will contain 111 trials, and last 777.0 seconds (12.95 mins)
Settings:

        Sessions: 1
        Blocks per session: 2
        Trials per block: 111
        Trials for each cond per block: 25.0
        Number of null trials per block: 11
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 1554.0 seconds
        Total duration: 25.9 minutes
        Total volumes: 1111 volumes
        Total duration of block: 777.0 seconds = 12.95 min
        Total number of volumes necessary for 1 block: 1 + 555 = 556 + warm-up pulses



In [84]:
# compuate all possile combinations of conditions
cond1, comb2final, comb3final, comb4final = gen_conditions(verbose=False)
    
# check if this is the right place to save the designs 
print('You are about to save these designs in: ' + os.getcwd())
print('Do not include /designs_MSIT in path, this is added automatically')
print('Check that this is correct')

# Ask if user wants to continue
ans=[]
while ans not in ['y','yes','Yes','n','no','No']:
    ans = input('Continue? (y or n): ')

# stop code if in wrong directory
if ans in ['n','no','No']:
    os.chdir(input('input path here: '))

# Create direcotry if it does not exist
save_dir = './designs_MSIT'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# How many sessions do you want?
session_nr = [1]
    
# add extra for loop for multiple sessions
for num_sess in session_nr:
    # loop through all subjects and generate designs
    for subby in range(1,pNums+1):

        # generate designs + onsets
        des = gen_onsets_and_design(TR=TR, nTRs=nTRs, nSub=subby, nConds=nConds, nBlocks=n_blocks, nTrialBlocks=n_not_null_trials,
                                  jitters=jitters, cond1=cond1, comb2final=comb2final, 
                                  comb3final=comb3final, comb4final=comb4final, verbose=True)

        pseudode_design = []

        # put designs through psuedorandomizer block by block 
        for blocky in des["block"].unique():
            print(blocky)
            this_subset = des.loc[des["block"]==blocky]

            updated_design = add_pseudorandom_null_trials(Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run(), 
                                                          min_row=round(n_null_trials*0.6), max_row=round(n_null_trials*0.6), min_n_rows_separate=round(n_null_trials*0.6), 
                                                          n_null_trials=n_null_trials, null_column_name='null_trial')

            updated_design['block'] = blocky
            pseudode_design.append(updated_design)

        design = pd.concat(pseudode_design, ignore_index=True)

        #add_jitter_to_null = np.tile(jitters, int((n_null_trials*n_blocks)/4))
        add_jitter_to_null = np.tile(jitters, int((len(design.loc[design.isnull().any(axis=1),'fix_circ_1'])*n_blocks)/4))
        np.random.shuffle(add_jitter_to_null)
        
        jitts_to_use = add_jitter_to_null[0:len(design.loc[design.isnull().any(axis=1),'fix_circ_1'])]

        print(len(design.loc[design.isnull().any(axis=1),'fix_circ_1']))
        print(len(jitts_to_use))

        design.loc[design.isnull().any(axis=1),'fix_circ_1'] =  jitts_to_use
        design.loc[design.isnull().any(axis=1),'ITI'] = (trial_len - stim_dur - timing_dur - jitts_to_use).round(3) 

        #design.loc[design.isnull().any(axis=1),'fix_circ_1'] = 0.75 # set all null trials jitter to 0.75s
        design.loc[design.isnull().any(axis=1),'stimulus_dur'] = stim_dur # 
        design.loc[design.isnull().any(axis=1),'feedback_timing_dur'] = timing_dur # 
        design.loc[design.isnull().any(axis=1),'feedback_cor_dur'] = feed_dur # 
        design['n_trs'] = nTRs
        #design.loc[design.isnull().any(axis=1),'ITI'] = 3.25 #

        design_prac = Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run().head(25)
        prac_jitts = np.random.choice(jitters,size=25)
        design_prac['block'] = 1
        design_prac['jitter'] = prac_jitts
        design_prac['feedback_timing_dur'] = 0.5
        design_prac['feedback_cor_dur'] = 0.5 
        design_prac['n_trs'] = nTRs
        design_prac['fix_circ_1'] = design_prac['jitter']
        design_prac['ITI'] = (trial_len - design_prac['feedback_timing_dur'][0] - design_prac['feedback_cor_dur'][0] - prac_jitts).round(3)

        fn = 'sub-' + str(subby).zfill(3) + '_ses-' + str(num_sess) + '_tr-' + str(TR) + '_design_task-MSIT'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

        fn_prac = 'sub-p' + str(subby).zfill(3) + '_ses-' + str(num_sess) + '_tr-' + str(TR) + '_design_task-MSIT'
        print(fn_prac)
        design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

You are about to save these designs in: /Users/scotti/surfdrive/Shared/NTNU_fMRI_tasks/tasks_for_at_NTNU/IMCN-MSIT
Do not include /designs_MSIT in path, this is added automatically
Check that this is correct
Continue? (y or n): y
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-001_ses-1_tr-1.4_design_task-MSIT
sub-p001_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-002_ses-1_tr-1.4_design_task-MSIT
sub-p002_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-003_ses-1_tr-1.4_design_task-MSIT
sub-p003_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-004_ses-1_tr-1.4_design_task-MSIT
sub-p004_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null tria

Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-045_ses-1_tr-1.4_design_task-MSIT
sub-p045_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-046_ses-1_tr-1.4_design_task-MSIT
sub-p046_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-047_ses-1_tr-1.4_design_task-MSIT
sub-p047_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-048_ses-1_tr-1.4_design_task-MSIT
sub-p048_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials
22
22
sub-049_ses-1_tr-1.4_design_task-MSIT
sub-p049_ses-1_tr-1.4_design_task-MSIT
1
Adding null trials
Finished adding null trials
2
Adding null trials
Finished adding null trials

# Generate pilot design under subject number 666

In [ ]:
# Define parameters for 666 debug design
s_num = 666
TR = TR               # same as defined above
n_blocks = n_blocks   # same as defined above
jitters = jitters     # same as defined above
n_trials = 96         # doesn't matter how many, because I will only take 5 from anyway
nTRs = 5
nConds = 4
n_sessions = 1          # number of sessions
stim_dur = 1.6         # stimulis presentation duration
timing_dur = 0.4         # timing feedback duration
feed_dur = 0.0          # correctness feedback duration (practice only)

# work out settings
block_length = None
num_trials = None
trial_len = nTRs*TR
p_null_trials = 0.1

# Work out what jitters times to use based on TR
jitters = [0.5, 1.0, 1.5, 2.0]

In [17]:
# session_nr is defined in main desgin script above
for num_sess in session_nr:

    # generate designs + onsets
    des = gen_onsets_and_design(TR=TR, nSub=s_num, nConds=4, nBlocks=n_blocks, nTrialBlocks=n_trials,
                              jitters=jitters, cond1=cond1, comb2final=comb2final, 
                              comb3final=comb3final, comb4final=comb4final, verbose=True)

    pseudode_design = []
    # put designs through psuedorandomizer block by block 
    for blocky in des["block"].unique():
        print(blocky)
        this_subset = des.loc[des["block"]==blocky]

        pseudo_subset = Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run()
        pseudode_design.append(pseudo_subset.head(5))

    design = pd.concat(pseudode_design, ignore_index=True)

    design_prac = Pseudorandomizer(this_subset.head(5), max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run()
    design_prac['block'] = 1
    design_prac['jitter'] = np.random.choice(jitters,size=5)
    design_prac['feedback_timing_dur'] = 0.5
    design_prac['feedback_cor_dur'] = 0.5 # 

    fn = 'sub-' + str(s_num).zfill(3) + '_ses-' + str(num_sess) + '_design_task-MSIT'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

    fn_prac = 'sub-p' + str(s_num).zfill(3) + '_ses-' + str(num_sess) + '_design_task-MSIT'
    print(fn_prac)
    design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')


1
sub-666_ses-1_design_task-MSIT
sub-p666_ses-1_design_task-MSIT
1
sub-666_ses-2_design_task-MSIT
sub-p666_ses-2_design_task-MSIT


# generate pilot MRI session designs

In [9]:
save_dir = '/Users/scotti/surfdrive/Shared/NTNU_fMRI_tasks/IMCN-MSIT/designs_MSIT'

In [16]:
subby = 666
n_blocks_pil = 1
n_trials = 48
nConds = 4
TR = 1.4
nTRs = 5
pilot_null_trials = 0
trial_len = TR * nTRs
stim_dur = 1
timing_dir = 0.4
feed_dur = 0.0
session_nr = 1

# compuate all possile combinations of conditions
cond1, comb2final, comb3final, comb4final = gen_conditions(verbose=False)
 
jitters = [.5,1.0,1.5,2.0]
    
# generate designs + onsets
des = gen_onsets_and_design(TR=TR, nTRs=nTRs, nSub=subby, nConds=nConds, nBlocks=n_blocks_pil, nTrialBlocks=n_trials,
                          jitters=jitters, cond1=cond1, comb2final=comb2final, 
                          comb3final=comb3final, comb4final=comb4final, verbose=True)

pseudode_design = []

# put designs through psuedorandomizer block by block 
for blocky in des["block"].unique():
    print(blocky)
    this_subset = des.loc[des["block"]==blocky]

#     updated_design = add_pseudorandom_null_trials(Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run(), min_row=4, max_row=4, min_n_rows_separate=7, 
#                             n_null_trials=pilot_null_trials-1, null_column_name='null_trial')
    
    updated_design = Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run()

    updated_design['block'] = blocky
    pseudode_design.append(updated_design)

#         pseudode_design.append(add_pseudorandom_null_trials(Pseudorandomizer(this_subset, max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run(), min_row=4, max_row=4, min_n_rows_separate=7, 
#                                 n_null_trials=n_null_trials, null_column_name='null_trial'))

#design = pd.concat(pseudode_design, ignore_index=True)

design = pd.concat(pseudode_design).reset_index(drop=True)


design['subject'] = subby

add_jitter_to_null = np.tile(jitters, int(pilot_null_trials*n_blocks_pil/4+2))[:pilot_null_trials*n_blocks_pil]
np.random.shuffle(add_jitter_to_null)

print(len(design.loc[design.isnull().any(axis=1),'fix_circ_1']))
print(len(add_jitter_to_null))

design.loc[design.isnull().any(axis=1),'fix_circ_1'] = add_jitter_to_null # 
design.loc[design.isnull().any(axis=1),'ITI'] = (trial_len - stim_dur - feed_dur - add_jitter_to_null).round(2) #

#design.loc[design.isnull().any(axis=1),'fix_circ_1'] = 0.75 # set all null trials jitter to 0.75s
design.loc[design.isnull().any(axis=1),'stimulus_dur'] = stim_dur # 
#design['null_trial'] = 0
design.loc[design.isnull().any(axis=1),'null_trial'] = 1
design.loc[design.isnull().any(axis=1),'feedback_cor_dur'] = 0
design.loc[design.isnull().any(axis=1),'feedback_timing_dur'] = 0
design.loc[design.isnull().any(axis=1),'jitter'] = design.loc[design.isnull().any(axis=1),'fix_circ_1']
#design.loc[design.isnull().any(axis=1),'feedback_dur'] = feed_dur # 
#design.loc['feedback_dur'] = feed_dur
#design.loc['feedback_cor_dur'] = feed_dur
#design.loc[design.isnull().any(axis=1),'ITI'] = 3.25 #

design = design.round({'ITI':1})
design['n_trs'] = nTRs

#design = design[:100]

design_prac = Pseudorandomizer(this_subset.head(25), max_identical_iters={'stimuli': 1, 'condition': 3, 'unique': 3}).run()
design_prac['block'] = 1
design_prac['jitter'] = np.random.choice(jitters,size=25)
design_prac['feedback_cor_dur'] = 0.4
design_prac['null_trial'] = 0 
design_prac['fix_circ_1'] = design_prac['jitter']
design_prac['n_trs'] = nTRs
    
fn = 'sub-' + str(subby).zfill(3) + '_ses-' + str(session_nr) + '_tr-' + str(TR) + '_design_task-MSIT'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')
    
fn_prac = 'sub-p' + str(subby).zfill(3) + '_ses-' + str(session_nr) + '_tr-' + str(TR) + '_design_task-MSIT'
print(fn_prac)
design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

1
0
0
sub-666_ses-1_tr-1.4_design_task-MSIT
sub-p666_ses-1_tr-1.4_design_task-MSIT


In [32]:
round((trial_len - stim_dur - feed_dur - add_jitter_to_null),2)

TypeError: type numpy.ndarray doesn't define __round__ method

In [37]:
design.round({'ITI':1})

,subject,stimuli,condition,jitter,unique,block,fix_circ_1,stimulus_dur,feedback_cor_dur,feedback_timing_dur,ITI,null_trial
0,444,331,4,2.0,1,1,2.0,1.6,0.0,0.4,2.4,0
1,444,002,2,1.5,2,1,1.5,1.6,0.0,0.4,2.9,0
2,444,113,3,2.0,3,1,2.0,1.6,0.0,0.4,2.4,0
3,444,311,4,1.5,3,1,1.5,1.6,0.0,0.4,2.9,0
4,444,113,3,1.0,3,1,1.0,1.6,0.0,0.4,3.4,0
5,444,133,3,1.0,1,1,1.0,1.6,0.0,0.4,3.4,0
6,444,002,2,1.5,2,1,1.5,1.6,0.0,0.4,2.9,0
7,444,121,3,0.5,2,1,0.5,1.6,0.0,0.4,3.9,0
8,444,100,1,2.0,1,1,2.0,1.6,0.0,0.4,2.4,0
9,444,020,1,1.0,2,1,1.0,1.6,0.0,0.4,3.4,0


In [33]:
trial_len - stim_dur - feed_dur - add_jitter_to_null

array([3.3, 2.8, 3.8, 3.3, 3.8, 2.8, 4.3, 3.3, 2.8, 3.3, 4.3, 3.8, 2.8,
       3.8, 3.3, 4.3, 4.3, 4.3, 2.8, 3.8])